In [1]:
# import libaries
import torch
import torch.nn as nn
from tqdm import tqdm
from torch.utils.data import DataLoader
from sklearn.manifold import TSNE

from sklearn.manifold import TSNE
from matplotlib import pyplot as plt
from import_classifiers import *
from dataset_classes import *
from settings import *

dev = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

np.random.seed(13)

/home/opontorno/miniconda3/envs/mdlearn/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def plot_features(backbone_name):
    fig, axs = plt.subplots(1, 3, figsize=(15, 5))
    trans = get_trans(model_name=backbone_name)
    testing_dset = make_balanced(mydataset(datasets_path, guidance_path, for_basemodel=False, for_testing=True, transforms=trans))
    testloader = DataLoader(testing_dset, batch_size=1, shuffle=True, num_workers=0, drop_last=False)
    folders = ['bm-dm', 'bm-gan', 'bm-real']
    for folder in folders:
        i = folders.index(folder)
        model = backbone(backbone_name, finetuning=False, as_feature_extractor=True)
        state_dict_path = os.path.join(models_dir, folder, backbone_name+folder[2:]+'.pt')
        model.load_state_dict(torch.load(state_dict_path))
        data, labels = [], []
        for batch in testloader:
            img,label,_= batch
            code = model(img)
            code.to('cpu').detach().numpy()
            labels.append(label.detach().numpy())
            data.append(code)
        data, labels = np.array(data), np.array(labels)
        tsne = TSNE(n_components=2, verbose=0, perplexity=300, n_iter=1000, init='pca')
        tsne_res = tsne.fit_transform(data.squeeze())
        axs[i].scatter(x=tsne_res[:,0], y=tsne_res[:,1], c=labels)
        axs[i].set_title(f'{folder}')

In [3]:
plot_features('densenet161')

Using cache found in /home/opontorno/.cache/torch/hub/pytorch_vision_v0.10.0
/home/opontorno/miniconda3/envs/mdlearn/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/opontorno/miniconda3/envs/mdlearn/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


: 